In [ ]:
from os.path import expanduser, join
# 以下を、ダウンロードしたデモのディレクリを指すように、修正してください。
HTS_DEMO_ROOT = join(expanduser("~"), "", "HTS-demo_NIT-ATR503-M001")

In [ ]:
HTS_DEMO_ROOT

In [ ]:
! ./scripts/copy_from_htsdemo.sh $HTS_DEMO_ROOT

In [ ]:
import sys
print(sys.path)
#sys.path.remove('/usr/local/lib/python3.7/site-packages')
sys.path.append('/usr/local/.pyenv/versions/3.6.0/lib/python3.6/site-packages')
sys.path.append('/Users/kazuya_yufune/.pyenv/versions/3.6.0/lib/python3.6/site-packages')

In [ ]:
%pylab inline
rcParams["figure.figsize"] = (16,5)

from nnmnkwii.datasets import FileDataSource, FileSourceDataset
from nnmnkwii.datasets import PaddedFileSourceDataset, MemoryCacheDataset#これはなに？
from nnmnkwii.preprocessing import trim_zeros_frames, remove_zeros_frames
from nnmnkwii.preprocessing import minmax, meanvar, minmax_scale, scale
from nnmnkwii import paramgen
from nnmnkwii.io import hts
from nnmnkwii.frontend import merlin as fe
from nnmnkwii.postfilters import merlin_post_filter

from os.path import join, expanduser, basename, splitext, basename, exists
import os
from glob import glob
import numpy as np
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
import pyworld
import pysptk
import librosa
import librosa.display
import IPython
from IPython.display import Audio

## Data

In this demo we construct datasets from pre-computed linguistic/duration/acoustic features because computing features from wav/label files on-demand are peformance heavy, particulary for acoustic features. See the following python script if you are interested in how we extract features.

In [ ]:
DATA_ROOT = "./data/basic5000"#NIT-ATR503/"#
test_size = 0.01 # This means 480 utterances for training data
random_state = 1234

In [ ]:
! python ./scripts/prepare_features2.py $DATA_ROOT --use_phone_alignment --question_path="./data/questions_jp2.hed"

### Data specification

Almost same as Merlin's slt_arctic demo. The only difference is that frequency warping paramter `alpha` is set to 0.41, instead of 0.58. As far as I know 0.41 is the best parameter approximating mel-frequency axis for 16kHz-sampled audio signals.

In [ ]:
mgc_dim = 180#メルケプストラム次数　？？
lf0_dim = 3#対数fo　？？ なんで次元が３？
vuv_dim = 1#無声or 有声フラグ　？？
bap_dim = 15#発話ごと非周期成分　？？

duration_linguistic_dim = 438#question_jp.hed で、ラベルに対する言語特徴量をルールベースで記述してる
acoustic_linguisic_dim = 442#上のやつ+frame_features とは？？
duration_dim = 1
acoustic_dim = mgc_dim + lf0_dim + vuv_dim + bap_dim #aoustice modelで求めたいもの

fs = 48000
frame_period = 5
fftlen = pyworld.get_cheaptrick_fft_size(fs)
alpha = pysptk.util.mcepalpha(fs)
hop_length = int(0.001 * frame_period * fs)

mgc_start_idx = 0
lf0_start_idx = 180
vuv_start_idx = 183
bap_start_idx = 184

windows = [
    (0, 0, np.array([1.0])),
    (1, 1, np.array([-0.5, 0.0, 0.5])),
    (1, 1, np.array([1.0, -2.0, 1.0])),
]

use_phone_alignment = True
acoustic_subphone_features = "coarse_coding" if use_phone_alignment else "full" #とは？

### File data sources

We need to specify 1) where to find pre-computed features and 2) how to process them. In this case,

1. `collect_files` : Collects `.bin` files. External python script writes files in binary format. Also we split the files into train/test set.
2. `collect_features` : Just load from file by `np.fromfile`.

In [ ]:
class BinaryFileSource(FileDataSource):
    def __init__(self, data_root, dim, train):
        self.data_root = data_root
        self.dim = dim
        self.train = train
    def collect_files(self):
        files = sorted(glob(join(self.data_root, "*.bin")))
        #files = files[:4000]#[:len(files)-5] # last 5 is real testset
        train_files = files#[:-5]
        test_files = files[-5:]
        #train_files, test_files = train_test_split(files, test_size=test_size,
                                                  # random_state=random_state)
        if self.train:
            return train_files
        else:
            return test_files
    def collect_features(self, path):
        return np.fromfile(path, dtype=np.float32).reshape(-1, self.dim)

In [ ]:
X = {"duration":{}, "acoustic": {}}
Y = {"duration":{}, "acoustic": {}}
utt_lengths = {"duration":{}, "acoustic": {}}
for ty in [ "acoustic"]:
    for phase in ["train", "test"]:
        train = phase == "train"
        x_dim = duration_linguistic_dim if ty == "duration" else acoustic_linguisic_dim
        y_dim = duration_dim if ty == "duration" else acoustic_dim
        X[ty][phase] = FileSourceDataset(BinaryFileSource(join(DATA_ROOT, "X_{}".format(ty)),
                                                       dim=x_dim,
                                                       train=train))
        Y[ty][phase] = FileSourceDataset(BinaryFileSource(join(DATA_ROOT, "Y_{}".format(ty)),
                                                       dim=y_dim,
                                                       train=train))
        utt_lengths[ty][phase] = np.array([len(x) for x in X[ty][phase]], dtype=np.int)

In [ ]:
Y['acoustic']['train'][0].shape

In [ ]:
X['acoustic']['train'][1].shape

Then we can construct datasets for duration and acoustic models. We wil have

- X: Input (duration, acoustic) datasets
- Y: Target (duration, acoustic) datasets

Note that dataset itself doesn't keep features in memory. It loads features on-demand while iteration or indexing.

For mini-batch sequenceial training (which should be drastically faster than batch_size=1), later we will use [PackedSequence](http://pytorch.org/docs/master/nn.html#torch.nn.utils.rnn.PackedSequence) in PyTorch. For this, re-create datasets with `PaddedFileSourceDataset`.

In [ ]:
"""
for ty in [ "acoustic"]:
    for phase in ["train", "test"]:
        train = phase == "train"
        x_dim = duration_linguistic_dim if ty == "duration" else acoustic_linguisic_dim
        y_dim = duration_dim if ty == "duration" else acoustic_dim
        X[ty][phase] = FileSourceDataset(BinaryFileSource(join(DATA_ROOT, "X_{}".format(ty)),
                                                       dim=x_dim,
                                                       train=train))
        Y[ty][phase] = FileSourceDataset(BinaryFileSource(join(DATA_ROOT, "Y_{}".format(ty)),
                                                       dim=y_dim,
                                                       train=train), )
"""

In [ ]:
X['acoustic']['train'][0]

### Utterance lengths

Let's see utterance lengths histrogram.

In [ ]:
print("Total number of utterances:", len(utt_lengths["acoustic"]["train"]))
print("Total number of frames:", np.sum(utt_lengths["acoustic"]["train"]))
hist(utt_lengths["acoustic"]["train"], bins=64);

### How data look like?

Pick an utterance from training data and visualize its features.

## ここ本気でわからん

In [ ]:
X_min = {}
X_max = {}
Y_mean = {}
Y_var = {}
Y_scale = {}

for typ in ["acoustic"]:
    X_min[typ], X_max[typ] = minmax(X[typ]["train"], utt_lengths[typ]["train"])
    Y_mean[typ], Y_var[typ] = meanvar(Y[typ]["train"], utt_lengths[typ]["train"])
    Y_scale[typ] = np.sqrt(Y_var[typ])

Linguistic features should be clear with normalization as shown below.

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from tqdm import tnrange, tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [ ]:
join(DATA_ROOT+'/mora_index', "*.csv")

In [ ]:
import pandas as pd

In [ ]:
mora_index_lists = ['data/basic5000/mora_index/mora_index_' + '0'*(4-len(str(i+1)))  + str(i+1) + '.csv' for i in range(5000)]
mora_index_lists = mora_index_lists#[:len(mora_index_lists)-5] # last 5 is real testset

mora_index_lists_for_model = [np.loadtxt(path).reshape(-1) for path in mora_index_lists]



#train_mora_index_lists, test_mora_index_lists = train_test_split(mora_index_lists_for_model, test_size=test_size,
#                                                  random_state=random_state)

train_mora_index_lists = mora_index_lists_for_model[:5000]
#test_mora_index_lists = mora_index_lists_for_model[4000:]


In [ ]:
device='cpu'

In [ ]:
train_index, test_index_ = train_test_split(list(range(4995)), test_size=0.01, random_state =1234)

In [ ]:
random_validation_indices = np.random.choice(test_index_, 4, replace = False)

In [ ]:
def gen_parameters(y_predicted):
    # Number of time frames
    T = y_predicted.shape[0]
    
    # Split acoustic features
    mgc = y_predicted[:,:lf0_start_idx]
    lf0 = y_predicted[:,lf0_start_idx:vuv_start_idx]
    plt.plot(lf0[:, 0])
    plt.show()
    #lf0 = Y['acoustic']['train'][90][:, lf0_start_idx:vuv_start_idx]
    #lf0 = np.zeros(lf0.shape)
    vuv = y_predicted[:,vuv_start_idx]

    plt.show()
    bap = y_predicted[:,bap_start_idx:]
    
    # Perform MLPG
    ty = "acoustic"
    mgc_variances = np.tile(Y_var[ty][:lf0_start_idx], (T, 1))#np.tile(Y_var[ty][:lf0_start_idx], (T, 1))
    mgc = paramgen.mlpg(mgc, mgc_variances, windows)
    lf0_variances = np.tile(Y_var[ty][lf0_start_idx:vuv_start_idx], (T,1))
    lf0 = paramgen.mlpg(lf0, lf0_variances, windows)
    bap_variances = np.tile(Y_var[ty][bap_start_idx:], (T, 1))
    bap = paramgen.mlpg(bap, bap_variances, windows)
    
    return mgc, lf0, vuv, bap
def gen_waveform(y_predicted, do_postfilter=False):  
    y_predicted = trim_zeros_frames(y_predicted)
        
    # Generate parameters and split streams
    mgc, lf0, vuv, bap = gen_parameters(y_predicted)
    
    if do_postfilter:
        mgc = merlin_post_filter(mgc, alpha)
        
    spectrogram = pysptk.mc2sp(mgc, fftlen=fftlen, alpha=alpha)
    aperiodicity = pyworld.decode_aperiodicity(bap.astype(np.float64), fs, fftlen)
    f0 = lf0.copy()
    f0[vuv < 0.5] = 0
    f0[np.nonzero(f0)] = np.exp(f0[np.nonzero(f0)])
    
    generated_waveform = pyworld.synthesize(f0.flatten().astype(np.float64),
                                            spectrogram.astype(np.float64),
                                            aperiodicity.astype(np.float64),
                                            fs, frame_period)
    return generated_waveform

In [ ]:
class VAE(nn.Module):
    def __init__(self, bidirectional=True, num_layers=1, z_dim=1):
        super(VAE, self).__init__()
        self.num_layers = num_layers
        self.num_direction =  2 if bidirectional else 1
        self.z_dim = z_dim
        self.lstm1 = nn.LSTM(acoustic_linguisic_dim+acoustic_dim, 400, 1 if num_layers==2 and (z_dim == 32 or z_dim==2) else num_layers , bidirectional=bidirectional, dropout=dropout)#入力サイズはここできまる
        self.fc21 = nn.Linear(self.num_direction*400, z_dim)
        self.fc22 = nn.Linear(self.num_direction*400, z_dim)
        ##ここまでエンコーダ
        
        self.lstm2 = nn.LSTM(acoustic_linguisic_dim+z_dim, 400, num_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc3 = nn.Linear(self.num_direction*400, acoustic_dim)

    def encode(self, linguistic_f, acoustic_f, mora_index):
        x = torch.cat([linguistic_f, acoustic_f], dim=1)
        out, hc = self.lstm1(x.view( x.size()[0],1, -1))
        nonzero_indices = torch.nonzero(mora_index.view(-1).data).squeeze()
        out = out[nonzero_indices]
        del nonzero_indices
        
        h1 = F.relu(out)

        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z, linguistic_features, mora_index, z_dsiabled=False, z0 = True):
        
        z_tmp = torch.tensor([[0]*self.z_dim]*linguistic_features.size()[0], dtype=torch.float32, requires_grad=True).to(device)
        count = 0
        prev_index = 0
        for i, mora_i in enumerate(mora_index):
            if mora_i == 1:
                z_tmp[prev_index:i] = z[count]
                prev_index = i
                count += 1
                
        if z_dsiabled:
            tmp = 0 if z0 else 1
            z_tmp = torch.tensor([[tmp]*self.z_dim]*linguistic_features.size()[0], dtype=torch.float32).to('cpu')
                

        
        x = torch.cat([linguistic_features, z_tmp.view(-1, self.z_dim)], dim=1).view(linguistic_features.size()[0], 1, -1)
        h3, (h, c) = self.lstm2(x)
        h3 = F.relu(h3)
        
        return self.fc3(h3)#torch.sigmoid(self.fc3(h3))

    def forward(self, linguistic_features, acoustic_features, mora_index, z_dsiabled = False, z0=True):
        mu, logvar = self.encode(linguistic_features, acoustic_features, mora_index)
        z = self.reparameterize(mu, logvar)
        
        return self.decode(z, linguistic_features, mora_index, z_dsiabled = z_dsiabled, z0 = z0), mu, logvar
    

In [ ]:
import wave

In [ ]:
test_index = 798#0#
#プロットするときはアクセントラベルの方がいいかも
#f0使いたいならタイムフレーム軸で、zは線形補完

#x_train = minmax_scale(x_train, x_min, x_max, feature_range=(0.01, 0.99))
#y_train = scale(y_train, y_mean, y_scale)


def check_reconstruction(test_index, z0=False):
    with torch.no_grad():
        x_train = X['acoustic']['train'][test_index]
        y_train = Y['acoustic']['train'][test_index]
        mora_i = train_mora_index_lists[test_index]
        x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
        y_train = torch.from_numpy(y_train).type(torch.FloatTensor)
        mora_i = torch.from_numpy(mora_i).type(torch.FloatTensor)
        mora_nonzero_index = torch.nonzero(mora_i.view(-1).data).squeeze()


        reconstruction, mu, logvar = model(x_train, y_train, mora_i)

        nonzero_indices = torch.nonzero(mora_i.view(-1).data).squeeze()
        prev_i = 0

        #言語特徴量、音響特徴量、mora_index
        true_accent = Y['acoustic']['train'][test_index][:, lf0_start_idx]#[np.where(train_mora_index_lists[test_index]>0)][:, lf0_start_idx]
        pred_accent = reconstruction.view(-1, 199).detach().numpy()[:, lf0_start_idx]#[np.where(train_mora_index_lists[test_index]>0)][:, lf0_start_idx]
        pred_accent = (pred_accent - true_accent.mean()) / true_accent.std()
        true_accent = (true_accent -true_accent.mean()) / true_accent.std()

        true_accent_hl = np.loadtxt('data/basic5000/accents/accents_' + '0'*(4-len(str(test_index)))  + str(test_index) + '.csv')

        """
        pred = mu.detach().numpy().reshape(-1)
        plt.plot(true_accent_hl, label='true accent')
        plt.plot(pred, label='z')
        plt.xlabel('mora_index')
        plt.legend()
        plt.show()
        #print('相関係数')
        #print(np.corrcoef([pred,true_accent_hl ])[0][1])


        pred_for_plot = torch.tensor([0]*pred_accent.shape[0], dtype=torch.float32)
        prev_index = 0

        for i, z in enumerate(mora_nonzero_index):
            pred_for_plot[prev_index:int(z)] = torch.tensor(pred[i])
            prev_index = int(z)

        pred_for_plot = pred_for_plot.numpy()
        plt.plot(true_accent, label='real lf0')
        plt.plot(pred_accent, label='pred lf0')
        #plt.plot(pred_for_plot, label='z')
        #plt.errorbar(range(logvar.size()[0]), mu.detach().numpy().reshape(-1), yerr = logvar.detach().numpy().reshape(-1), label='z')
        plt.legend()
        plt.xlabel('time frame')
        plt.show()
        """




        #zが普通




        print("z normal")
        IPython.display.display(Audio(gen_waveform(reconstruction.view(-1, 199).detach().numpy(), True), rate=fs))
        if z0:
            reconstruction_, mu_, logvar_ = model(x_train, y_train, mora_i, z_dsiabled=True)
            print("z0")
            IPython.display.display(Audio(gen_waveform(reconstruction_.view(-1, 199).detach().numpy(), True), rate=fs))
            print("z1")
            reconstruction_, mu_, logvar_ = model(x_train, y_train, mora_i, z_dsiabled=True, z0=False)
            IPython.display.display(Audio(gen_waveform(reconstruction_.view(-1, 199).detach().numpy(), True), rate=fs))

            IPython.display.display(Audio(gen_waveform(Y['acoustic']['train'][test_index], True), rate=fs))


    return mu, logvar, 


In [ ]:
num_layers = 2
z_dim = 1
dropout=0.3 
model = VAE(num_layers=num_layers, z_dim=z_dim).to('cpu')

In [ ]:
#junk = 'junk_models/vae_mse_0.01kld_z_changed_losssum_batchfirst_20.pth'
paths = sorted(glob('optuna0612/{}layers_zdim{}_model*'.format(num_layers, z_dim)))
epochs = []
for path in paths:
    epoch = int(path[path.index('l_')+2:path.index('.')])
    epochs.append(epoch)
    
epoch_index = np.array(epochs).argmax()

#model.load_state_dict(torch.load('optuna0612/{}layers/zdim{}/vae.pth'.format(num_layers, z_dim), map_location=torch.device('cpu') ))
model.load_state_dict(torch.load(paths[epoch_index], map_location=torch.device('cpu') ))

In [ ]:
def mse(A, B) :
    return (np.square(A - B)).mean()

In [ ]:
from tqdm import tqdm

In [ ]:
test_indices = [i*20+1 for i in range(250)]
import random
random.seed(12345)#12345
randomli_sampled_test = random.sample(test_indices, 10)

se = 0
f0_e = 0
for test_index in tqdm(test_indices):
    x_train = X['acoustic']['train'][test_index]
    y_train = Y['acoustic']['train'][test_index]
    mora_i = train_mora_index_lists[test_index]
    x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
    y_train = torch.from_numpy(y_train).type(torch.FloatTensor)
    mora_i = torch.from_numpy(mora_i).type(torch.FloatTensor)
    
    
    reconstruction, mu, logvar  = model(x_train, y_train, mora_i)
    
    del x_train
    del y_train
    del mora_i
    
    se += mse(Y['acoustic']['train'][test_index], reconstruction.detach().numpy())
    f0_e += mse(Y['acoustic']['train'][test_index][:, lf0_start_idx:vuv_start_idx], reconstruction.detach().numpy().reshape(-1, 199)[:, lf0_start_idx:vuv_start_idx])

In [ ]:
print(se)
print(f0_e)

In [ ]:
randomli_sampled_test

In [ ]:
#2layers_zdim8
mus = []
logvars = []
num_layers = 3
z_dim = 8
dropout=0.3 
model = VAE(num_layers=num_layers, z_dim=z_dim).to('cpu')
model.load_state_dict(torch.load('accent_models/{}layers/zdim{}/vae.pth'.format(num_layers, z_dim), map_location=torch.device('cpu') ))

for index in randomli_sampled_test:
    mu, logvar = check_reconstruction(index, z0=False)
    mus.append(mu.detach().numpy())
    logvars.append(logvar.detach().numpy())

In [ ]:
#2layers_zdim32
mus = []
logvars = []
num_layers = 2
z_dim = 4
dropout=0.3 
model = VAE(num_layers=num_layers, z_dim=z_dim).to('cpu')
model.load_state_dict(torch.load('accent_models/{}layers/zdim{}/vae.pth'.format(num_layers, z_dim), map_location=torch.device('cpu') ))

for index in randomli_sampled_test:
    mu, logvar = check_reconstruction(index, z0=False)
    mus.append(mu.detach().numpy())
    logvars.append(logvar.detach().numpy())

In [ ]:
zd_nl_combination = [[1,1], [1, 2], [1, 4], [1, 8], [1, 32], [2, 1], [2, 2], [2, 4], [2, 8], [2, 32], [3, 8]]
f0_errors = []
rmse_all = []

for nl, zd in zd_nl_combination:
    f0_error, mse_all = error_calculator(zd, nl, test_indices)
    f0_errors.append(f0_error)
    rmse_all.append(mse_all)

In [ ]:
for a, b in zip(rmse_all, zd_nl_combination):
    print(a, b)

In [ ]:
for a, b in zip(f0_errors, zd_nl_combination):
    print(a, b)

In [ ]:
def rmse(a, b):
    return np.sqrt(np.mean((a - b) ** 2))

def calc_lf0_rmse(natural, generated, lf0_idx, vuv_idx):
    idx = (natural[:, vuv_idx] * (generated[:, vuv_idx] >= 0.5)).astype(bool)
    return rmse(natural[idx, lf0_idx], generated[idx, lf0_idx]) * 1200 / np.log(2) 

In [ ]:
def error_calculator(z_dim, num_layers, test_indices):
    model = VAE(num_layers=num_layers, z_dim=z_dim).to('cpu')
    model.load_state_dict(torch.load('accent_models/{}layers/zdim{}/vae.pth'.format(num_layers, z_dim), map_location=torch.device('cpu')))
    
    f0_errors = []
    mse_all = []
    
    for test_index in tqdm(test_indices):
        with torch.no_grad():
            x_train = X['acoustic']['train'][test_index]
            y_train = Y['acoustic']['train'][test_index]
            mora_i = train_mora_index_lists[test_index]
            x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
            y_train = torch.from_numpy(y_train).type(torch.FloatTensor)
            mora_i = torch.from_numpy(mora_i).type(torch.FloatTensor)
            reconstruction, mu, logvar = model(x_train, y_train, mora_i)
            y_train = y_train.numpy().reshape(-1, 199)
            reconstruction = reconstruction.numpy().reshape(-1, 199)
            
        f0_errors.append(calc_lf0_rmse(y_train, reconstruction, lf0_start_idx, vuv_start_idx))
        mse_all.append(rmse(y_train, reconstruction))
        
    return np.array(f0_errors).mean(), np.array(mse_all).mean()


In [ ]:
#0609は81と41??
#良い 1, 81, 4641
#悪い 41, 0, 4041
test_index = 581
mu, logvar = check_reconstruction(test_index, z0=True)

In [ ]:

test_accent = np.loadtxt('data/basic5000/accents/accents_' + '0'*(4-len(str(test_index)))+ str(test_index) + '.csv')
if z_dim == 1:
    
    pred_z = mu.detach().numpy().reshape(-1)
    
    xlabel = ['te', 'tsu', 'no', 'ka', 'a', 'te ', 'N', 'ga', 'yo', 'o', 'ro', 'cl', 'pa', 'ta', 'i', 'ri', 'ku ', 'ni', 'o ', 'ri ', 'ta ']
    #xlabel = ['mi', 'zu', 'o', 'ma', 're', 'e', 'shi', 'a', 'ka', 'ra', 'ka ', 'wa', 'na ', 'ke', 're ', 'ba', 'na  ', 'ra ', 'na', 'i', 'no', 'de', 'su']
    plt.title('basic' + '0'*(4-len(str(test_index+1)))+ str(test_index+1))
    plt.plot(xlabel, test_accent, label='true_accent_label')
    plt.errorbar(xlabel, pred_z, yerr= torch.exp(0.5*logvar).detach().numpy().reshape(-1), label='pred z')
    plt.ylabel('accent_label')
    plt.xlabel('mora index')
    plt.legend()
    plt.show()
    
    plt.scatter(test_accent, pred_z)
    plt.xlabel('true accent label')
    plt.ylabel('predicted z')
    plt.show()

    print('相関係数')
    print(np.corrcoef(test_accent, pred_z)[0][1])

In [ ]:
mus_tmp = []
labels = []
test_accents = [np.loadtxt('data/basic5000/accents/accents_' + '0'*(4-len(str(test_index)))+ str(test_index) + '.csv') for test_index in randomli_sampled_test]
mus = np.array(mus)
for i in range(mus.shape[0]):
    mu_tmp = mus[i].reshape(-1, z_dim)
    for j in range(mu_tmp.shape[0]):
        mus_tmp.append(mu_tmp[j])
        labels.append(test_accents[i][j])

mu_data = np.array(mus_tmp)



In [ ]:
len(test_accents)

In [ ]:
mu_data = np.array(mu_data)
mu_data.shape

In [ ]:
if z_dim ==1:
    plt.scatter(labels, mu_data)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
#主成分分析の実行
pca = PCA()
pca.fit(mu_data)
# データを主成分空間に写像
feature = pca.transform(mu_data)

In [ ]:
plt.scatter(feature[:, 0], feature[:, 1], c=labels, )
plt.title("{}layers_zdim{}".format(num_layers, z_dim))
plt.colorbar()
plt.show()

In [ ]:
X_reduced = TSNE(n_components=2, random_state=0).fit_transform(mu_data)

In [ ]:
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels)
plt.show()

In [ ]:
import random
indices = [random.randint(0, 5000) for i in range(10)]

In [ ]:
#1, 4 90, 798がいいzz感じ
test_index = 798
mu, logvar = check_reconstruction(test_index)

for index in random_validation_indices:
    print(index)
    mu, logvar = check_reconstruction(index)
    print('\n\n\n\n\n')

In [ ]:
zs = []
for test_index in indices:
    #test_index = 0
    mu, logvar = check_reconstruction(test_index)
    zs.append(mu.detach().numpy())

In [ ]:
zs = np.array(zs).reshape(-1)

In [ ]:
tmp = np.array([[0, 0]])

for i in range(10):
    tmp_x = zs[i].reshape(-1, 2)
    tmp = np.concatenate([tmp, tmp_x],)

In [ ]:
tmp.shape

In [ ]:
plt.scatter(tmp[:, 0], tmp[:, 1])

In [ ]:
plt.xlabel('mora_index')
std = torch.exp(0.5*logvar[:, 1])
plt.errorbar(range(logvar.size()[0]), mu, yerr = std.detach().numpy().reshape(-1), label='mu ± logvar')
plt.ylabel('z')
plt.show()

In [ ]:
pred_for_plot

In [ ]:
mu

# train_loader -> [X[acoustic][train],  Y[acoustic][train], mora_index] 
## ここの順番づけが終わればひと段落

In [ ]:
device='cpu'
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x.view(-1), x.view(-1, ), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

#train_loader -> [X[acoustic][train],  Y[acoustic][train], mora_index] ここの順番づけが終わればひと段落

func_tensor = np.vectorize(torch.from_numpy)

X_acoustic_train = [torch.from_numpy(X['acoustic']['train'][i]) for i in range(len(X['acoustic']['train']))] 
Y_acoustic_train = [torch.from_numpy(Y['acoustic']['train'][i]) for i in range(len(Y['acoustic']['train']))]
train_mora_index_lists = [torch.tensor(train_mora_index_lists[i]) for i in range(len(train_mora_index_lists))]

X_acoustic_test = [torch.from_numpy(X['acoustic']['test'][i]) for i in range(len(X['acoustic']['test']))]
Y_acoustic_test = [torch.from_numpy(Y['acoustic']['test'][i]) for i in range(len(Y['acoustic']['test']))]
test_mora_index_lists = [torch.tensor(test_mora_index_lists[i]) for i in range(len(test_mora_index_lists))]

train_loader = [[X_acoustic_train[i], Y_acoustic_train[i], train_mora_index_lists[i]] for i in range(len(train_mora_index_lists))]
test_loader = [[X_acoustic_test[i], Y_acoustic_test[i], test_mora_index_lists[i]] for i in range(len(test_mora_index_lists))]


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        for j in range(3):
            data[j] = data[j].to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data[0], data[1], data[2])
        loss = loss_function(recon_batch, data[1], mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / 1))
    
    return train_loss


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data, in enumerate(test_loader):
            for j in range(3):
                data[j] = data[j].to(device)
            recon_batch, mu, logvar = model(data[0], data[1], data[2])
            test_loss += loss_function(recon_batch, data[1], mu, logvar).item()
            """
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)
            """

    test_loss /= 1
    print('====> Test set loss: {:.4f}'.format(test_loss))
    
    return test_loss

In [ ]:
for i, x in enumerate(test_mora_index_lists):
    if x.size()[0] >= 1900:
        print(x.size())
        print(i)

In [ ]:
for i, x in enumerate(X_acoustic_test):
    if x.size()[0] >= 1900:
        print(x.size())
        print(i)

In [ ]:
X['acoustic']['train'][0].shape

In [ ]:
loss_list = []
test_loss_list = []
num_epochs = 5

for epoch in range(1, num_epochs + 1):
    loss = train(epoch)
    test_loss = test(epoch)
    print(loss)
    print(test_loss)

    print('epoch [{}/{}], loss: {:.4f} test_loss: {:.4f}'.format(
        epoch + 1,
        num_epochs,
        loss,
        test_loss))

    # logging
    loss_list.append(loss)
    test_loss_list.append(test_loss)

# save the training model
np.save('loss_list.npy', np.array(loss_list))
np.save('test_loss_list.npy', np.array(test_loss_list))
torch.save(model.state_dict(), 'vae.pth')

## Train

### Configurations

Network hyper parameters and training configurations (learning rate, weight decay, etc).

In [ ]:
num_hidden_layers = 3
hidden_size = 512

batch_size = 8#発話

n_workers = 2
pin_memory = True
nepoch = 10
lr = 0.002
weight_decay = 1e-6
use_cuda = torch.cuda.is_available()
print(use_cuda)

### Trainining loop

Our RNN predicts output feature sequence given a input feature sequence, so we need to feed our data to network in sequence-wise manner. This is pretty easy. We can just use `MemoryCacheDataset` that supports utterancew-wise iteration and has cache functionality to avoid file re-loading.

In [ ]:
def train_rnn(model, optimizer, X, Y, X_min, X_max, Y_mean, Y_scale,
          utt_lengths, cache_size=1000):
    if use_cuda:
        model = model.cuda()
        
    X_train, X_test = X["train"], X["test"]
    Y_train, Y_test = Y["train"], Y["test"]
    train_lengths, test_lengths = utt_lengths["train"], utt_lengths["test"]
    
    # Sequence-wise train loader
    X_train_cache_dataset = MemoryCacheDataset(X_train, cache_size)
    Y_train_cache_dataset = MemoryCacheDataset(Y_train, cache_size)
    train_dataset = PyTorchDataset(X_train_cache_dataset, Y_train_cache_dataset, train_lengths,
                                  X_min, X_max, Y_mean, Y_scale)
    train_loader = data_utils.DataLoader(
        train_dataset, batch_size=batch_size, num_workers=n_workers, shuffle=True)
    
    # Sequence-wise test loader
    X_test_cache_dataset = MemoryCacheDataset(X_test, cache_size)
    Y_test_cache_dataset = MemoryCacheDataset(Y_test, cache_size)
    test_dataset = PyTorchDataset(X_test_cache_dataset, Y_test_cache_dataset, test_lengths,
                                 X_min, X_max, Y_mean, Y_scale)
    test_loader = data_utils.DataLoader(
        test_dataset, batch_size=batch_size, num_workers=n_workers, shuffle=True)
    
    dataset_loaders = {"train": train_loader, "test": test_loader}
        
    # Training loop
    criterion = nn.MSELoss()
    model.train()
    print("Start utterance-wise training...")
    loss_history = {"train": [], "test": []}
    for epoch in tnrange(nepoch):
        for phase in ["train", "test"]:
            running_loss = 0
            for x, y, lengths in dataset_loaders[phase]:
                # Sort by lengths . This is needed for pytorch's PackedSequence
                sorted_lengths, indices = torch.sort(lengths.view(-1), dim=0, descending=True)
                sorted_lengths = sorted_lengths.long().numpy()
                # Get sorted batch
                x, y = x[indices], y[indices]
                # Trim outputs with max length
                y = y[:, :sorted_lengths[0]]
                
                # Init states
                h, c = model.init_hidden(len(sorted_lengths))
                if use_cuda:
                    x, y = x.cuda(), y.cuda()
                if use_cuda:
                    h, c = h.cuda(), c.cuda()
                x, y = Variable(x), Variable(y)
                optimizer.zero_grad()
                
                # Do apply model for a whole sequence at once
                # no need to keep states
                y_hat = model(x, sorted_lengths, h, c)
                loss = criterion(y_hat, y)
                if phase == "train":
                    loss.backward()
                    optimizer.step()
                running_loss += loss.item()
            loss_history[phase].append(running_loss / (len(dataset_loaders[phase])))
        
    return loss_history

### Define models

In [ ]:
models = {}
length = {'duration': 531, 'acoustic': 535}
for typ in ["duration", "acoustic"]:
    models[typ] = MyRNN(length[typ],
                            hidden_size, Y[typ]["train"][0].shape[-1],
                            num_hidden_layers, bidirectional=True)
    print("Model for {}\n".format(typ), models[typ])

In [ ]:
models['duration'].load_state_dict(torch.load('models_basic5000/model_duration', map_location=torch.device('cpu')))
models['acoustic'].load_state_dict(torch.load("models_basic5000/model_acoustic", map_location=torch.device('cpu')))

### Training Duration model

In [ ]:
ty = "duration"
optimizer = optim.Adam(models[ty].parameters(), lr=lr, weight_decay=weight_decay)
loss_history = train_rnn(models[ty], optimizer, X[ty], Y[ty],
                     X_min[ty], X_max[ty], Y_mean[ty], Y_scale[ty], utt_lengths[ty])

In [ ]:
plot(loss_history["train"], linewidth=2, label="Train loss")
plot(loss_history["test"], linewidth=2, label="Test loss")
legend(prop={"size": 16})

### Training acoustic model

In [ ]:
ty = "acoustic"
optimizer = optim.Adam(models[ty].parameters(), lr=lr, weight_decay=weight_decay)
loss_history = train_rnn(models[ty], optimizer, X[ty], Y[ty],
                     X_min[ty], X_max[ty], Y_mean[ty], Y_scale[ty], utt_lengths[ty])

In [ ]:
plot(loss_history["train"], linewidth=2, label="Train loss")
plot(loss_history["test"], linewidth=2, label="Test loss")
legend(prop={"size": 16})

## Test

Let's see how our network works.

### Parameter generation utilities

Almost same as DNN text-to-speech synthesis. The difference is that we need to give initial hidden states explicitly.

In [ ]:
binary_dict, continuous_dict = hts.load_question_set("./data/questions_jp2.hed")

def gen_parameters(y_predicted):
    # Number of time frames
    T = y_predicted.shape[0]
    
    # Split acoustic features
    mgc = y_predicted[:,:lf0_start_idx]
    lf0 = y_predicted[:,lf0_start_idx:vuv_start_idx]
    #lf0 = np.zeros(lf0.shape)
    vuv = y_predicted[:,vuv_start_idx]
    bap = y_predicted[:,bap_start_idx:]
    
    # Perform MLPG
    ty = "acoustic"
    mgc_variances = np.tile(Y_var[ty][:lf0_start_idx], (T, 1))
    mgc = paramgen.mlpg(mgc, mgc_variances, windows)
    lf0_variances = np.tile(Y_var[ty][lf0_start_idx:vuv_start_idx], (T,1))
    lf0 = paramgen.mlpg(lf0, lf0_variances, windows)
    bap_variances = np.tile(Y_var[ty][bap_start_idx:], (T, 1))
    bap = paramgen.mlpg(bap, bap_variances, windows)
    
    return mgc, lf0, vuv, bap

def gen_waveform(y_predicted, do_postfilter=False):  
    y_predicted = trim_zeros_frames(y_predicted)
        
    # Generate parameters and split streams
    mgc, lf0, vuv, bap = gen_parameters(y_predicted)
    
    if do_postfilter:
        mgc = merlin_post_filter(mgc, alpha)
        
    spectrogram = pysptk.mc2sp(mgc, fftlen=fftlen, alpha=alpha)
    aperiodicity = pyworld.decode_aperiodicity(bap.astype(np.float64), fs, fftlen)
    f0 = lf0.copy()
    f0[vuv < 0.5] = 0
    f0[np.nonzero(f0)] = np.exp(f0[np.nonzero(f0)])
    
    generated_waveform = pyworld.synthesize(f0.flatten().astype(np.float64),
                                            spectrogram.astype(np.float64),
                                            aperiodicity.astype(np.float64),
                                            fs, frame_period)
    return generated_waveform
    
def gen_duration(hts_labels, duration_model):
    # Linguistic features for duration
    duration_linguistic_features = fe.linguistic_features(hts_labels,
                                               binary_dict, continuous_dict,
                                               add_frame_features=False,
                                               subphone_features=None).astype(np.float32)

    # Apply normalization
    ty = "duration"
    duration_linguistic_features = minmax_scale(duration_linguistic_features, 
                                       X_min[ty], X_max[ty], feature_range=(0.01, 0.99))
    
    # Apply models
    duration_model = duration_model.cpu()
    duration_model.eval()
    
    #  Apply model
    x = Variable(torch.from_numpy(duration_linguistic_features)).float()
    try:
        duration_predicted = duration_model(x).data.numpy()
    except:
        h, c = duration_model.init_hidden(batch_size=1)
        xl = len(x)
        x = x.view(1, -1, x.size(-1))
        duration_predicted = duration_model(x, [xl], h, c).data.numpy()
        duration_predicted = duration_predicted.reshape(-1, duration_predicted.shape[-1])
                     
    # Apply denormalization
    duration_predicted = duration_predicted * Y_scale[ty] + Y_mean[ty]
    duration_predicted = np.round(duration_predicted)
    
    # Set minimum state duration to 1
    duration_predicted[duration_predicted <= 0] = 1
    hts_labels.set_durations(duration_predicted)
    
    return hts_labels    


def test_one_utt(hts_labels, duration_model, acoustic_model, post_filter=True):
    # Predict durations
    duration_modified_hts_labels = gen_duration(hts_labels, duration_model)
    
    # Linguistic features
    linguistic_features = fe.linguistic_features(duration_modified_hts_labels, 
                                                  binary_dict, continuous_dict,
                                                  add_frame_features=True,
                                                  subphone_features=acoustic_subphone_features)
    # Trim silences
    indices = duration_modified_hts_labels.silence_frame_indices()
    linguistic_features = np.delete(linguistic_features, indices, axis=0)

    # Apply normalization
    ty = "acoustic"
    linguistic_features = minmax_scale(linguistic_features, 
                                       X_min[ty], X_max[ty], feature_range=(0.01, 0.99))
    
    # Predict acoustic features
    acoustic_model = acoustic_model.cpu()
    acoustic_model.eval()
    x = Variable(torch.from_numpy(linguistic_features)).float()
    try:
        acoustic_predicted = acoustic_model(x).data.numpy()
    except:
        h, c = acoustic_model.init_hidden(batch_size=1)
        xl = len(x)
        x = x.view(1, -1, x.size(-1))
        acoustic_predicted = acoustic_model(x, [xl], h, c).data.numpy()
        acoustic_predicted = acoustic_predicted.reshape(-1, acoustic_predicted.shape[-1])
             
    # Apply denormalization
    acoustic_predicted = acoustic_predicted * Y_scale[ty] + Y_mean[ty]
    
    return gen_waveform(acoustic_predicted, post_filter)    

In [ ]:
Y

In [ ]:
def gen_parameters(y_predicted):
    # Number of time frames
    T = y_predicted.shape[0]
    
    # Split acoustic features
    mgc = y_predicted[:,:lf0_start_idx]

    #mgc = np.zeros(mgc.shape)
    lf0 = y_predicted[:,lf0_start_idx:vuv_start_idx]
    #lf0 = np.ones(lf0.shape) * 4
    vuv = y_predicted[:,vuv_start_idx]#声帯を使わない囁き声になる
    #vuv = np.zeros(vuv.shape[0])
    bap = y_predicted[:,bap_start_idx:]
    #bap = np.zeros(bap.shape)
    
    # Perform MLPG
    ty = "acoustic"
    mgc_variances = np.tile(Y_var[ty][:lf0_start_idx], (T, 1))
    mgc = paramgen.mlpg(mgc, mgc_variances, windows)
    lf0_variances = np.tile(Y_var[ty][lf0_start_idx:vuv_start_idx], (T,1))
    lf0 = paramgen.mlpg(lf0, lf0_variances, windows)
    bap_variances = np.tile(Y_var[ty][bap_start_idx:], (T, 1))
    bap = paramgen.mlpg(bap, bap_variances, windows)
    
    return mgc, lf0, vuv, bap

In [ ]:
IPython.display.display(Audio(gen_waveform(Y['acoustic']['train'][0], True), rate=fs))

### Listen generated audio

Generated audio samples with Merlin's slt_full_demo are attached. You can compare them below.

In [ ]:
print("MyRNN")
label_path = './data/basic5000/test_phone_align/BASIC5000_4501.lab'
hts_labels = hts.load(label_path)
waveform = test_one_utt(hts_labels, models["duration"], models["acoustic"])
#wavfile.write(join(save_dir, basename(wav_path1)), rate=fs, data=waveform)
IPython.display.display(Audio(waveform, rate=fs))

In [ ]:
test_label_paths = sorted(glob(join(DATA_ROOT, "test_phone_align", "*.lab")))
ffn_generated_wav_files = sorted(glob(join("./generated/jp-01-tts/*.wav")))
hts_generated_wav_files = sorted(glob(join("./generated/hts_nit_atr503_2mix/*.wav")))

# Save generated wav files for later comparizon
save_dir = join("./generated/jp-02-tts")
if not exists(save_dir):
    os.makedirs(save_dir)
    
for label_path  in test_label_paths:

    print("MyRNN")
    hts_labels = hts.load(label_path)
    waveform = test_one_utt(hts_labels, models["duration"], models["acoustic"])
    #wavfile.write(join(save_dir, basename(wav_path1)), rate=fs, data=waveform)
    IPython.display.display(Audio(waveform, rate=fs))


### TTS using OpenJTalk frontend

Using the OpenJTalk text processing frontend, we can generate speech for any input text. In this notebook, we use https://github.com/r9y9/pyopenjtalk to use OpenJtalk frontend functionality.

注意: 現時点では、nnmnkwiiとpyopenjtalkの開発版が必要です

In [ ]:
sys.path.append('../pyopenjtalk')

In [ ]:
sys.path.append('Users/kazuya_yufune/Desktop/猿渡研究室/チュートリアル/nnmnkwii_gallery/notebooks/pyopenjtalk/')

In [ ]:
import pyopenjtalk

for idx, text in enumerate([
    "こんにちは。本日は、お越しいただき誠にありがとうございます。",
    "拙者は、サムライでございまする。",
    "最後まで読んでいただき、本当にありがとうございます。",
    "フィードバックがあれば、ぜひお教えくださいませ。",
    "音声合成の実現には、自然言語処理、機械学習、音声信号処理など、複数の分野に渡る知識が必要です",
    "このチュートリアルですべてをカバーしているわけではありませんが、少しでも学習の助けになれば幸いです。",
    "ありがとうございました！",
]):
    _, labels = pyopenjtalk.run_frontend(text)
    hts_labels = hts.load(lines=labels)
    print(idx, text)
    waveform = test_one_utt(hts_labels, models["duration"], models["acoustic"])
    IPython.display.display(Audio(waveform, rate=fs))

In [ ]:
bunbo = np.sqrt(0.1) * ( np.pi / 2 -   np.arctan(1/np.sqrt(0.1))) + 1


In [ ]:
1/bunbo

In [ ]:
gamma = 0.1
root_gamma = np.sqrt(gamma)
p1 = 100
p2 = 1

In [ ]:
bunbo1 = root_gamma*(np.pi/2 - np.arctan(1/root_gamma) ) + np.sqrt(p2*gamma / p1) * (np.pi/2 - np.arctan(np.sqrt(p1/p2/gamma))) + 1

In [ ]:
1/ bunbo1

In [ ]:
bunbo2 = root_gamma*(np.pi/2 - np.arctan(1/root_gamma) ) + np.sqrt(p1*gamma / p2) * (np.pi/2 - np.arctan(np.sqrt(p2/p1/gamma))) + 1

In [ ]:
1 / bunbo2

In [ ]:
bunbo1_4 = root_gamma*(np.pi/2 - np.arctan(1/root_gamma) ) + np.sqrt(p2*gamma / p1) * (np.pi/2 - np.arctan(np.sqrt(p2/p1/gamma))) + 1

In [ ]:
bunbo2_4 = root_gamma*(np.pi/2 - np.arctan(1/root_gamma) ) + np.sqrt(p1*gamma / p2) * (np.pi/2 - np.arctan(np.sqrt(p1/p2/gamma))) + 1

In [ ]:
1/bunbo1_4

In [ ]:
1 / bunbo2_4